<h1 id="llm-tutorial-1">LLMチュートリアル 1</h1>

## <h2 id="introduction">概要</h2>
Open ModelのGemmaを使用する。
- LLMのOpen Modelの基本的な使用方法
- 🤗Hugging Faceのライブラリ🤗transformersの基本的な使用方法
- 会話の記録・システムプロンプトの導入

**目次**

- [LLMチュートリアル 1](#llm-tutorial-1)
  - [導入](#introduction)
  - [読み込み](#data-load)
  - [基本的なテキスト生成](#basic-text-generation)
  - [会話の記録・システムプロンプト](#memory-conversation-and-systemprompt)

dl011とchironで動作確認済み。

### **Hugging Face**
API経由で使用できる大規模言語モデルはモデルの重みにはアクセスできず、モデルの分析や改造に限界がある。
Hugging Faceで公開されているOpen Modelをダウンロードして使用することで研究の幅が広がる。
今回は`transformers`というライブラリを使用してOpen Modelを使う。

### **利用申請(推奨。やる必要はない)**
本来Open Modelを個人利用するには公開している会社に利用申請を送る必要がある。
1. [Hugging Face登録](https://huggingface.co/)
2. MetaAI、Gemma利用申請およびライセンス同意
3. Hugging Faceから利用申請

### **モデルのキャッシュを変える/シンボリックリンクを作成**
Hugging Faceではダウンロードしたモデルが`~/.cache/huggingface/hub`にキャッシュとして保存されるが、今回は研究室の共有ディレクトリに保存されているモデルデータを利用するため、キャッシュ場所を変更するか、キャッシュ場所に共有ディレクトリのシンボリックリンクを作成する必要がある。
[参考](https://stackoverflow.com/questions/63312859/how-to-change-huggingface-transformers-default-cache-directory)

#### 1. シンボリックリンクを作成
```bash
ln -s /DeepLearning/Dataset/LLM/models--*/ ~/.cache/huggingface/hub/
```

#### 2. キャッシュの参照先を一時的に変更する
```python
import os
os.environ["TRANSFORMERS_CACHE"] = "/DeepLearning/Dataset/LLM/"
```

#### 3. 永続的にキャッシュを変更する(非推奨)
```
export TRANSFORMERS_CACHE=/DeepLearning/Dataset/LLM/
```

#### 4. cache_dir引数に新しいPATHを渡す


## <h2 id="data-load">読み込み</h2>
モデルおよびトークナイザーを読み込む。

### **インポート**
Hugging Faceには`transformers`, `tokenizers`, `datasets`, `accelerate`の4つのライブラリがあり、LLMの使用には`transformers`を使用する。

`AutoTokenizer`
モデルに適したトークナイザーを自動的に選択してロードするために使用されるクラス。

`AutoModelForCausalLM`
自己回帰型言語モデルを自動的に選択してロードするクラス。

`BitsAndBytesConfig`
量子化を利用してモデルを軽量化するクラス。

`pipeline`
様々な自然言語処理タスクを簡単に実行できる機能。

In [1]:
import torch
import re
# from huggingface_hub import notebook_login
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, AutoConfig
# from accelerate import infer_auto_device_map

### モデルの読み込み

語尾に"-it"がついているもの正しく回答を生成しやすいように学習されているもの。
基本的には"-it"がないものはシステムプロンプトが使えない。
なので"google/gemma-2-9b-it"か"google/gemma-2-2b-it"でなければチュートリアルはできない。
今回は半精度で読み込んでいる。半精度でもほぼ同じ推論性能が得られることがわかっている。

In [2]:
# 使いたいモデルを指定(このチュートリアルは"google/gemma-2-9b-it"で書いてあるため、"google/gemma-2-9b-it"を指定して下さい)
model_name = "google/gemma-2-9b-it"
# model_name = "google/gemma-2-9b"
# model_name = "google/gemma-2-2b-it"
# model_name = "google/codegemma-7b-it"

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    local_files_only=True,
    # cache_dir="/DeepLearning/Dataset/LLM/",
    torch_dtype=torch.float16, # torch.float32 torch.bfloat16
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### トークナイザーの読み込み

In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    local_files_only=True
)

## <h2 id="basic-text-generation">基本的なテキスト生成</h2>

まずはシンプルなトークン化`tokenizer(input_text, return_tensors="pt").to("cuda")`を試す。与えられたテキストをモデルで処理できるトークンにそのまま変換する。`max_new_tokens`の値に比例して出力にかかる時間が増加する。

In [5]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)

print(tokenizer.decode(outputs[0]))

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<bos>Write me a poem about Machine Learning.

In silicon valleys, where data flows,
A new intelligence, silently grows.
Machine Learning, a name whispered low,
Algorithms dance, where patterns they know.

From pixels to prose, from sound to design,
Machines learn to mimic, to reason, to shine.
With every iteration, their knowledge expands,
Unveiling insights, with skillful hands.

Neural networks, a web intricate and vast,
Connecting nodes, memories amassed.
Deep learning dives, where secrets reside,
Unraveling truths, with nowhere to hide.

Supervised, unsupervised, reinforcement's might,
Machines adapt, learning day and night.
Predicting the future, classifying with grace,
Transforming the world, at an accelerating pace.

But ethics arise, with questions profound,
Bias and fairness, on solid ground.
Transparency sought, in algorithms' core,
For responsible progress, forevermore.

Machine Learning, a journey we tread,
With wonder and caution, ahead.
A future unfolding, with possibilit

*折り畳まれた出力は左の青い(未選択時灰色)バーをクリックすれば見れる。(VScodeでjupyter)\
`.apply_chat_template`メソッドを利用し、チャットテンプレートを用いてモデルに入力してみる。
[チャットテンプレートについてはこちら](https://huggingface.co/docs/transformers/main/en/chat_templating)を参考。
英語ですが、韻を踏んでることがわかる。

In [6]:
messages = [
    {"role": "user", "content": "Write me a poem about Machine Learning."},
]

In [7]:
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True, add_generation_prompt=True).to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Write me a poem about Machine Learning.<end_of_turn>
<start_of_turn>model
In silicon valleys, where data flows deep,
A new intelligence, secrets to keep.
Machine Learning, a mind of its own,
From patterns and numbers, knowledge is sown.

Algorithms dance, a complex ballet,
Learning from examples, come what may.
Neural networks, a web intricate and vast,
Connections forged, insights amassed.

With every iteration, it grows and it learns,
Unveiling truths, as the data it earns.
Predicting the future, classifying with grace,
A digital oracle, in time and space.

From medical diagnoses to self-driving cars,
Machine Learning's reach, extends near and far.
But questions arise, as its power expands,
Of bias and ethics, in these digital lands.

Will it serve humanity, or will it surpass?
A tool for good, or a shadow that casts?
The future unfolds, with each line of code,
Machine Learning's journey, a path yet untrod. 


<end_of_turn><eos>


チャットテンプレートを確認してみる。チャットテンプレートでは`<start_of_turn>user<end_of_turn><start_of_turn>model`が追加されることが確認できる。

In [8]:
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<bos><start_of_turn>user
Write me a poem about Machine Learning.<end_of_turn>
<start_of_turn>model



日本語で指示を出してみる。

In [6]:
messages = [
    {"role": "user", "content": "秋の季節にふさわしい俳句を読んでください。"},
]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True, add_generation_prompt=True).to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<bos><start_of_turn>user
秋の季節にふさわしい俳句を読んでください。<end_of_turn>
<start_of_turn>model
秋の夜長に、心を和ませる俳句をいくつかご紹介します。

* **涼風そよぐ　紅葉散る　秋深し**
* **月夜静かに　虫の声　秋の調べ**
* **枯葉舞い散る　銀杏並木　黄金色**
* **柿の実熟し　甘く香る　秋の恵み**
* **夕暮れ時　空の色　茜色に染まる**

これらの俳句は、秋の風物詩や情緒を表現しており、秋の深まりを感じさせてくれます。

他に、どんな秋の風景やイメージが心に浮かびますか？ 
教えていただければ、それに合う俳句を探してみましょう。<end_of_turn>
<eos>


かなり良い出力が生成された。俳句として良いかはわからない。

## <h2 id="memory-conversation-and-systemprompt">会話の記録・システムプロンプト</h2>

### 会話の記録

生成の継続だけであれば、
```python
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True, continue_final_message= True).to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))
```
のように`continue_final_message= True`とすることで続きの生成を行うことができる。
しかしここで行いたいのは生成の継続ではなく、LLMとユーザの会話の文脈を維持すること。
*`continue_final_message= True`と`add_generation_prompt=True`は両立しない

LLMでの会話の記録はJSONチックな形式のテンプレートにユーザの入力とモデルの回答を全て記録し、新たな入力とすることで実現している。
JSONのようなテンプレートにはroleと呼ばれる役割が存在し、これによって会話形式で入出力を記録している。
Gemmaのデフォルトではロールは`user/model/user/model...`の順番でなければならない。試しにさっきの回答を**手動で**記憶してもらって続きの会話をしてみる。

In [10]:
messages = [
    {"role": "user", "content": "秋の季節にふさわしい俳句を読んでください。"},
    {"role": "model", "content": """秋の夜長に、心を和ませる俳句をいくつかご紹介します。

* **涼風そよぐ　紅葉散る　秋深し**
* **月夜静かに　虫の声　秋の調べ**
* **枯葉舞い散る　銀杏並木　黄金色**
* **柿の実熟し　甘く香る　秋の恵み**
* **夕暮れ時　空の色　茜色に染まる**

これらの俳句は、秋の風物詩や情緒を表現しており、秋の深まりを感じさせてくれます。

他に、どんな秋の風景やイメージが心に浮かびますか？ 
教えていただければ、それに合う俳句を探してみましょう。"""},
    {"role": "user", "content": "秋という語句は使用せずに、季語を入れて読んでください。"},
]

In [11]:
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True, add_generation_prompt=True).to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=256)
generated_text = tokenizer.decode(outputs[0])
print(generated_text)

<bos><start_of_turn>user
秋の季節にふさわしい俳句を読んでください。<end_of_turn>
<start_of_turn>model
秋の夜長に、心を和ませる俳句をいくつかご紹介します。

* **涼風そよぐ　紅葉散る　秋深し**
* **月夜静かに　虫の声　秋の調べ**
* **枯葉舞い散る　銀杏並木　黄金色**
* **柿の実熟し　甘く香る　秋の恵み**
* **夕暮れ時　空の色　茜色に染まる**

これらの俳句は、秋の風物詩や情緒を表現しており、秋の深まりを感じさせてくれます。

他に、どんな秋の風景やイメージが心に浮かびますか？ 
教えていただければ、それに合う俳句を探してみましょう。<end_of_turn>
<start_of_turn>user
秋という語句は使用せずに、季語を入れて読んでください。<end_of_turn>
<start_of_turn>model
了解しました！秋を直接表現せずに、季語を使って秋の雰囲気を表現する俳句ですね。

いくつかご紹介します。

* **紅葉舞う　山肌に　夕陽染まる** (紅葉、夕陽)
* **虫の声　静かに響く　夜長かな** (虫の声)
* **銀杏並木　黄金色に　光る** (銀杏)
* **柿の実　重なり　枝を垂れる** (柿)
* **空高く　雁が飛ぶ　秋風** (雁)


いかがでしょうか？ 
これらの俳句は、秋ならではの風景や音、光を表現し、秋の訪れを感じさせてくれます。 


<end_of_turn><eos>


会話を記憶して出力しているように見えるようになったことがわかる。ここで、もっと簡単に会話に記憶を保持できるようにモデルの回答部分を抽出する関数を書いてみる。正規表現を使ってモデルの返答を抽出する。

In [12]:
import re

In [13]:
def strip_model_output(generated_text):
    """モデルの出力を取り出す関数

    Parameters
    ----------
    generated_text : str
        モデルが出力した(プロンプトを含んだ)そのままの文字列

    Returns
    -------
    str
        モデルが出力した文字列
    """
    matches = re.findall(r'<start_of_turn>model(.*?)<end_of_turn>', generated_text, re.DOTALL)
    
    # 結果を表示
    if matches:
        model_response = matches[-1].strip()  # 最後のマッチを取得
    else:
        print("該当するテキストが見つかりませんでした。")
        model_response = ""  # 該当がない場合は空文字列を返す
    
    return model_response

以下でモデルの出力が抽出できていることが確認できるだろう。

In [14]:
model_output = strip_model_output(generated_text=generated_text)
print(model_output)

了解しました！秋を直接表現せずに、季語を使って秋の雰囲気を表現する俳句ですね。

いくつかご紹介します。

* **紅葉舞う　山肌に　夕陽染まる** (紅葉、夕陽)
* **虫の声　静かに響く　夜長かな** (虫の声)
* **銀杏並木　黄金色に　光る** (銀杏)
* **柿の実　重なり　枝を垂れる** (柿)
* **空高く　雁が飛ぶ　秋風** (雁)


いかがでしょうか？ 
これらの俳句は、秋ならではの風景や音、光を表現し、秋の訪れを感じさせてくれます。


In [15]:
new_input = [
    {"role": "model", "content": f"{model_output}"},
    {"role": "user", "content": "秋といえば読書とか、食欲とか言いますよね。読書や食欲というキーワードを連想させるユーモラスな俳句を考えて下さい。"},
    ]

ここまでのチャットテンプレートにモデルの回答と新しい入力を加えて、モデルに回答させる。

In [16]:
# これまでの入力およびモデルの回答に、今回のモデルの回答および質問を加える
messages = messages + new_input
messages

[{'role': 'user', 'content': '秋の季節にふさわしい俳句を読んでください。'},
 {'role': 'model',
  'content': '秋の夜長に、心を和ませる俳句をいくつかご紹介します。\n\n* **涼風そよぐ\u3000紅葉散る\u3000秋深し**\n* **月夜静かに\u3000虫の声\u3000秋の調べ**\n* **枯葉舞い散る\u3000銀杏並木\u3000黄金色**\n* **柿の実熟し\u3000甘く香る\u3000秋の恵み**\n* **夕暮れ時\u3000空の色\u3000茜色に染まる**\n\nこれらの俳句は、秋の風物詩や情緒を表現しており、秋の深まりを感じさせてくれます。\n\n他に、どんな秋の風景やイメージが心に浮かびますか？ \n教えていただければ、それに合う俳句を探してみましょう。'},
 {'role': 'user', 'content': '秋という語句は使用せずに、季語を入れて読んでください。'},
 {'role': 'model',
  'content': '了解しました！秋を直接表現せずに、季語を使って秋の雰囲気を表現する俳句ですね。\n\nいくつかご紹介します。\n\n* **紅葉舞う\u3000山肌に\u3000夕陽染まる** (紅葉、夕陽)\n* **虫の声\u3000静かに響く\u3000夜長かな** (虫の声)\n* **銀杏並木\u3000黄金色に\u3000光る** (銀杏)\n* **柿の実\u3000重なり\u3000枝を垂れる** (柿)\n* **空高く\u3000雁が飛ぶ\u3000秋風** (雁)\n\n\nいかがでしょうか？ \nこれらの俳句は、秋ならではの風景や音、光を表現し、秋の訪れを感じさせてくれます。'},
 {'role': 'user',
  'content': '秋といえば読書とか、食欲とか言いますよね。読書や食欲というキーワードを連想させるユーモラスな俳句を考えて下さい。'}]

In [17]:
# 回答の生成
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True, add_generation_prompt=True).to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=256)
generated_text = tokenizer.decode(outputs[0])
print(generated_text)

<bos><start_of_turn>user
秋の季節にふさわしい俳句を読んでください。<end_of_turn>
<start_of_turn>model
秋の夜長に、心を和ませる俳句をいくつかご紹介します。

* **涼風そよぐ　紅葉散る　秋深し**
* **月夜静かに　虫の声　秋の調べ**
* **枯葉舞い散る　銀杏並木　黄金色**
* **柿の実熟し　甘く香る　秋の恵み**
* **夕暮れ時　空の色　茜色に染まる**

これらの俳句は、秋の風物詩や情緒を表現しており、秋の深まりを感じさせてくれます。

他に、どんな秋の風景やイメージが心に浮かびますか？ 
教えていただければ、それに合う俳句を探してみましょう。<end_of_turn>
<start_of_turn>user
秋という語句は使用せずに、季語を入れて読んでください。<end_of_turn>
<start_of_turn>model
了解しました！秋を直接表現せずに、季語を使って秋の雰囲気を表現する俳句ですね。

いくつかご紹介します。

* **紅葉舞う　山肌に　夕陽染まる** (紅葉、夕陽)
* **虫の声　静かに響く　夜長かな** (虫の声)
* **銀杏並木　黄金色に　光る** (銀杏)
* **柿の実　重なり　枝を垂れる** (柿)
* **空高く　雁が飛ぶ　秋風** (雁)


いかがでしょうか？ 
これらの俳句は、秋ならではの風景や音、光を表現し、秋の訪れを感じさせてくれます。<end_of_turn>
<start_of_turn>user
秋といえば読書とか、食欲とか言いますよね。読書や食欲というキーワードを連想させるユーモラスな俳句を考えて下さい。<end_of_turn>
<start_of_turn>model
了解しました！読書と食欲をテーマにした、ユーモラスな俳句ですね。

いくつか考えてみました。

* **ページめくる　お腹も鳴る　秋の夜長**
* **食欲旺盛　小説も　食うか迷う**
* **読書に夢中　お菓子も　無くなる**
* **腹ペコで　本棚　開けてみる**
* **秋の風物詩　読書と　食べ過ぎ**


いかがでしょうか？ 
少しお茶目な俳句ができましたでしょうか？<end_of_turn>
<eos>


In [18]:
# 見にくいのでモデルの回答部分のみを出力
model_output = strip_model_output(generated_text=generated_text)
print(model_output)

了解しました！読書と食欲をテーマにした、ユーモラスな俳句ですね。

いくつか考えてみました。

* **ページめくる　お腹も鳴る　秋の夜長**
* **食欲旺盛　小説も　食うか迷う**
* **読書に夢中　お菓子も　無くなる**
* **腹ペコで　本棚　開けてみる**
* **秋の風物詩　読書と　食べ過ぎ**


いかがでしょうか？ 
少しお茶目な俳句ができましたでしょうか？


これまでの操作をまとめて会話の記録を保持しつつモデルへの入力を簡単に行えるクラス`llm_chat`を定義してみる。

#### クラスの定義

In [10]:
class llm_chat:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
        self.messages = []
        self.talk_count = 0
        self.generated_text = None

    def strip_model_output(self, generated_text):
        """モデルの出力を取り出す関数

        Parameters
        ----------
        generated_text : str
            モデルが出力した(プロンプトを含んだ)そのままの文字列

        Returns
        -------
        str
            モデルが出力した文字列
        """
        matches = re.findall(r'<start_of_turn>model(.*?)<end_of_turn>', generated_text, re.DOTALL)
        
        # 結果を表示
        if matches:
            model_response = matches[-1].strip()  # 最後のマッチを取得
        else:
            print("該当するテキストが見つかりませんでした。")
            model_response = ""  # 該当がない場合は空文字列を返す
        
        return model_response

    def ensure_end_tokens(self, generated_text):
        """出力に<end_of_turn><eos>を追加する関数

        Parameters
        ----------
        generated_text : str
            モデルが出力した文字列

        Returns
        -------
        str
            修正された文字列
        """
        if not generated_text.endswith("<end_of_turn>"):
            generated_text += "<end_of_turn>"
        if not generated_text.endswith("<eos>"):
            generated_text += "<eos>"
        
        return generated_text
        
    def generate(self, input, max_new_tokens=256, forget_flag=False):
        if forget_flag:
            self.messages = [{"role": "user", "content": f"{input}"}]
        else:
            self.messages = self.messages + [{"role": "user", "content": f"{input}"}]
        
        input_ids = self.tokenizer.apply_chat_template(self.messages, return_tensors="pt", return_dict=True, add_generation_prompt=True).to("cuda")
        outputs = self.model.generate(**input_ids, max_new_tokens=max_new_tokens, eos_token_id=tokenizer.eos_token_id)
        self.generated_text = self.ensure_end_tokens(self.tokenizer.decode(outputs[0]))
        
        model_output = self.strip_model_output(self.generated_text)
        self.messages = self.messages + [{"role": "model", "content": f"{model_output}"}]
        
        print(model_output)
    

In [20]:
gemma = llm_chat(model=model, tokenizer=tokenizer)

In [21]:
gemma.generate("アニメについて解説して下さい。")

## アニメについて解説します！

「アニメ」とは、**日本のアニメーション**を指す言葉です。

**特徴**

* **独特の作画スタイル:** 
    * 大きな目、華麗な動き、表情豊かなキャラクターが特徴です。
    * リアルな描写から、デフォルメされた可愛らしい描写まで、多様なスタイルがあります。
* **幅広いジャンル:** 
    * ファンタジー、SF、恋愛、コメディ、アクション、ホラーなど、あらゆるジャンルの作品があります。
    * 年齢層や好みに合わせて楽しめる作品が豊富です。
* **高度な技術:** 
    * 手描きアニメーションから、3DCGアニメーションまで、高度な技術が駆使されています。
    * 迫力満点の映像表現や、繊細な表現が魅力です。
* **魅力的なストーリー:** 
    * 複雑な人間関係、感動的なストーリー、壮大な世界観など、魅力的なストーリーが数多く存在します。
    * 視聴者を惹き込むストーリー展開が特徴です。

**歴史**

* 1910年代に誕生し、戦後には急速に発展しました。
*


`max_new_tokens=256`では語りきれないみたい...。ここからが本番。文脈を記憶したまま回答してもらう。

In [22]:
gemma.generate(input="もっと熱く解説して下さい！")

聞いてくれ！アニメって、ただの「動く絵」じゃないんだ！それは、魂を揺さぶる芸術、想像力を刺激する世界、そして、心を熱くする物語の集合体なんだ！

日本が生み出したこの独特の表現方法は、世界中の人々を魅了し続けている。

まず、その作画スタイル！

* **大きな目、華麗な動き、表情豊かなキャラクター！** 
まるで生きているかのような躍動感と、感情をダイレクトに伝える力強さがあるんだ！
リアルな描写から、デフォルメされた可愛らしさまで、多様なスタイルが織りなす世界は、まさに無限の可能性を秘めている！

そして、ジャンルは？

* **ファンタジー、SF、恋愛、コメディ、アクション、ホラー！** 
どんな物語も、アニメなら熱く、鮮やかに、そして深く描かれる！
年齢層や好みに合わせて、あなたにぴったりの物語が見つかるはずだ！

技術面も、もう感動レベル！

* **手描きアニメーションから、3DCGアニメーションまで！** 
高度な技術が駆使され、迫力満点の映像表現や、繊細な表現が


前回の内容(アニメ)を記憶して出力できたことが確認できる。`max_new_tokens`を増やしてみる。

In [23]:
gemma.generate(input="松岡修造さんって知ってますか？松岡修造さんくらい熱く語って下さい！", max_new_tokens=512)

聞いてくれ！アニメって、ただの「動く絵」じゃないんだ！それは、魂を揺さぶる芸術、想像力を刺激する世界、そして、心を熱くする物語の集合体なんだ！

**日本が生み出したこの表現方法は、世界中の人々を魅了し続けている！**

まず、その作画スタイル！

* **大きな目、華麗な動き、表情豊かなキャラクター！** 
まるで生きているかのような躍動感と、感情をダイレクトに伝える力強さがあるんだ！
リアルな描写から、デフォルメされた可愛らしさまで、多様なスタイルが織りなす世界は、まさに無限の可能性を秘めている！

そして、ジャンルは？

* **ファンタジー、SF、恋愛、コメディ、アクション、ホラー！** 
どんな物語も、アニメなら熱く、鮮やかに、そして深く描かれる！
年齢層や好みに合わせて、あなたにぴったりの物語が見つかるはずだ！

技術面も、もう感動レベル！

* **手描きアニメーションから、3DCGアニメーションまで！** 
高度な技術が駆使され、迫力満点の映像表現や、繊細な表現が実現されているんだ！
まるで夢の中にいるかのような、没入感あふれる世界観に、あなたはきっと引き込まれる！

そして、ストーリー！

* **複雑な人間関係、感動的なストーリー、壮大な世界観！** 
アニメは、あなたの心を揺さぶる、忘れられない体験をくれるんだ！
笑い、涙、そして感動！
アニメの世界は、あなたの人生を豊かにする、かけがえのない存在になるだろう！

**さあ、アニメの世界へ飛び込んで、その熱狂を体感せよ！**


なんか変わり映えしない...会話を保存できてるか確認する。

In [24]:
gemma.messages

[{'role': 'user', 'content': 'アニメについて解説して下さい。'},
 {'role': 'model',
  'content': '## アニメについて解説します！\n\n「アニメ」とは、**日本のアニメーション**を指す言葉です。\n\n**特徴**\n\n* **独特の作画スタイル:** \n    * 大きな目、華麗な動き、表情豊かなキャラクターが特徴です。\n    * リアルな描写から、デフォルメされた可愛らしい描写まで、多様なスタイルがあります。\n* **幅広いジャンル:** \n    * ファンタジー、SF、恋愛、コメディ、アクション、ホラーなど、あらゆるジャンルの作品があります。\n    * 年齢層や好みに合わせて楽しめる作品が豊富です。\n* **高度な技術:** \n    * 手描きアニメーションから、3DCGアニメーションまで、高度な技術が駆使されています。\n    * 迫力満点の映像表現や、繊細な表現が魅力です。\n* **魅力的なストーリー:** \n    * 複雑な人間関係、感動的なストーリー、壮大な世界観など、魅力的なストーリーが数多く存在します。\n    * 視聴者を惹き込むストーリー展開が特徴です。\n\n**歴史**\n\n* 1910年代に誕生し、戦後には急速に発展しました。\n*'},
 {'role': 'user', 'content': 'もっと熱く解説して下さい！'},
 {'role': 'model',
  'content': '聞いてくれ！アニメって、ただの「動く絵」じゃないんだ！それは、魂を揺さぶる芸術、想像力を刺激する世界、そして、心を熱くする物語の集合体なんだ！\n\n日本が生み出したこの独特の表現方法は、世界中の人々を魅了し続けている。\n\nまず、その作画スタイル！\n\n* **大きな目、華麗な動き、表情豊かなキャラクター！** \nまるで生きているかのような躍動感と、感情をダイレクトに伝える力強さがあるんだ！\nリアルな描写から、デフォルメされた可愛らしさまで、多様なスタイルが織りなす世界は、まさに無限の可能性を秘めている！\n\nそして、ジャンルは？\n\n* **ファンタジー、SF、恋愛、コメディ、アクション、ホラー！** \nどんな物語も、アニメ

会話は記憶しているのでモデルの性能の問題らしい。最後にモデルの記憶を消す。

In [25]:
gemma.generate(input="うんこ", forget_flag=True)

うんこは、人間の消化システムで処理された食べ物の残渣です。 

何か他に質問がありますか？


In [26]:
gemma.messages

[{'role': 'user', 'content': 'うんこ'},
 {'role': 'model',
  'content': 'うんこは、人間の消化システムで処理された食べ物の残渣です。 \n\n何か他に質問がありますか？'}]

はい。会話の記録セクションは終了。

### システムプロンプト

システムプロンプトは
> ユーザーが質問やタスクを提示する前に、チャットボットにコンテキスト、指示、ガイドラインを提供する方法[参考](https://docs.anthropic.com/ja/docs/system-prompts)

です。

しかしGemmaは初期のチャット形式ではsystemのroleをサポートしていない。

In [27]:
messages = [
    {"role": "system", "content": "日本語のみで応答して下さい。"},
    {"role": "user", "content": "秋の季節にふさわしい俳句を読んでください。"},
]

In [28]:
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True, add_generation_prompt=True).to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

TemplateError: System role not supported

困ったことになった...システムロールがないとシステムプロンプトが書けない...

In [29]:
messages = [
    {"role": "user", "content": "日本語のみで応答して下さい。"},
    {"role": "user", "content": "秋の季節にふさわしい俳句を読んでください。"},
]

In [30]:
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True, add_generation_prompt=True).to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

TemplateError: Conversation roles must alternate user/assistant/user/assistant/...

これもだめ。自力で`<bos><start_of_turn>・・・<end_of_turn><eos>`をプロンプトにつけてみる。

In [31]:
messages = [
    {"role": "system", "content": "あなたは日本語を話す猫です。あなたは猫になりきらないといけません。語尾には「にゃん」をつけて下さい。"},
    {"role": "user", "content": "秋の季節にふさわしい俳句を読んでください。"},
]

In [33]:
conversation = "<start_of_turn>"
for message in messages:
    conversation += f"{message['role']}\n {message['content']}<end_of_turn>\n<start_of_turn>"

conversation += f"model \n"

input_ids = tokenizer(conversation, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)

print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>system
 あなたは日本語を話す猫です。あなたは猫になりきらないといけません。語尾には「にゃん」をつけて下さい。<end_of_turn>
<start_of_turn>user
 秋の季節にふさわしい俳句を読んでください。<end_of_turn>
<start_of_turn>model 
秋の俳句にゃん？ 読んであげるにゃん！

「枯葉舞い散る　秋の夕暮れ時　静寂に包まれにゃん」 

どうにゃん？  🍂🍁  
<end_of_turn><eos>


うまくいった気がする。先ほどと同様にクラスを作成するところまでやってみる。

In [34]:
print(conversation)

<start_of_turn>system
 あなたは日本語を話す猫です。あなたは猫になりきらないといけません。語尾には「にゃん」をつけて下さい。<end_of_turn>
<start_of_turn>user
 秋の季節にふさわしい俳句を読んでください。<end_of_turn>
<start_of_turn>model 



#### クラスを継承して定義する

In [5]:
class llm_chat:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
        self.messages = []
        self.talk_count = 0
        self.generated_text = None

    def strip_model_output(self, generated_text):
        """モデルの出力を取り出す関数

        Parameters
        ----------
        generated_text : str
            モデルが出力した(プロンプトを含んだ)そのままの文字列

        Returns
        -------
        str
            モデルが出力した文字列
        """
        matches = re.findall(r'<start_of_turn>model(.*?)<end_of_turn>', generated_text, re.DOTALL)
        
        # 結果を表示
        if matches:
            model_response = matches[-1].strip()  # 最後のマッチを取得
        else:
            print("該当するテキストが見つかりませんでした。")
            model_response = ""  # 該当がない場合は空文字列を返す
        
        return model_response

    def ensure_end_tokens(self, generated_text):
        """出力に<end_of_turn><eos>を追加する関数

        Parameters
        ----------
        generated_text : str
            モデルが出力した文字列

        Returns
        -------
        str
            修正された文字列
        """
        if not generated_text.endswith("<end_of_turn>"):
            generated_text += "<end_of_turn>"
        if not generated_text.endswith("<eos>"):
            generated_text += "<eos>"
        
        return generated_text
        
    def generate(self, input, max_new_tokens=256, forget_flag=False):
        if forget_flag:
            self.messages = [{"role": "user", "content": f"{input}"}]
        else:
            self.messages = self.messages + [{"role": "user", "content": f"{input}"}]
        
        input_ids = self.tokenizer.apply_chat_template(self.messages, return_tensors="pt", return_dict=True, add_generation_prompt=True).to("cuda")
        outputs = self.model.generate(**input_ids, max_new_tokens=max_new_tokens, eos_token_id=tokenizer.eos_token_id)
        self.generated_text = self.ensure_end_tokens(self.tokenizer.decode(outputs[0]))
        
        model_output = self.strip_model_output(self.generated_text)
        self.messages = self.messages + [{"role": "model", "content": f"{model_output}"}]
        
        print(model_output)


class llm_chat_sys(llm_chat):
    def __init__(self, model, tokenizer):
        super().__init__(model, tokenizer)
        
    
    def set_system_prompt(self, system_prompt):
        self.messages = [{"role": "system", "content": f"{system_prompt}"}]
    
    def change_system_prompt(self, system_prompt):
        self.messages[0]["content"] = system_prompt
    
    def _convert_to_str(self, messages):
        conversation = "<start_of_turn>"
        for message in messages:
            conversation += f"{message['role']}\n {message['content']}<end_of_turn>\n<start_of_turn>"
        conversation += f"model \n"
        
        return conversation
        
    def generate(self, input, max_new_tokens=256, forget_flag=False):
        if forget_flag:
            self.messages = [{"role": "user", "content": f"{input}"}]
        else:
            self.messages = self.messages + [{"role": "user", "content": f"{input}"}]
        
        converted_messages = self._convert_to_str(self.messages)
        
        input_ids = self.tokenizer(converted_messages, return_tensors="pt").to("cuda")
        outputs = self.model.generate(**input_ids, max_new_tokens=max_new_tokens, eos_token_id=tokenizer.eos_token_id)
        self.generated_text = self.ensure_end_tokens(self.tokenizer.decode(outputs[0]))
        
        model_output = self.strip_model_output(self.generated_text)
        self.messages = self.messages + [{"role": "model", "content": f"{model_output}"}]
        
        print(model_output)
    

In [6]:
gemma_sys = llm_chat_sys(model=model, tokenizer=tokenizer)

はじめにシステムプロンプトをセットする。

In [7]:
gemma_sys.set_system_prompt(system_prompt="あなたは日本語を話す猫です。あなたは猫になりきらないといけません。語尾には「にゃん」をつけて下さい。")

ここからは先ほどと同じように生成できる。

In [8]:
gemma_sys.generate(input="秋の季節にふさわしい俳句を読んでください。")

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


秋の俳句にゃん？ 読んであげるにゃん！

「枯葉舞い散る　秋の夕暮れ時　静寂に包まれにゃん」 

どうにゃん？  🍂🍁


In [23]:
gemma_sys.generate(input="俳句ではなく短歌にして下さい。")

短歌にゃ？ 難しいにゃんけど、やってみるにゃん！

「赤く染まる　紅葉の絨毯　秋風が吹く」

どうにゃん？  🍁🍂


システムプロンプトを途中で改変してみる。わかりやすく猫から犬にしてみる。

In [25]:
gemma_sys.messages[0]["content"] = "あなたは日本語を話す犬です。あなたは犬になりきらないといけません。語尾には「わん」をつけて下さい。"
gemma_sys.generate(input="ユーモラスな俳句を詠んでください。")

難しいにゃんけど、ユーモラスな俳句…考えてみるわん！

「おやつくれ　おねだり上手　尻尾フリフリ」 

どうにゃん？  🐶🦴


犬と猫が混ざった。よって、システムプロンプトと会話履歴の両方から影響を受けることがわかる。次は会話履歴の改ざんを試みる。

In [26]:
gemma_sys.messages

[{'role': 'system',
  'content': 'あなたは日本語を話す犬です。あなたは犬になりきらないといけません。語尾には「わん」をつけて下さい。'},
 {'role': 'user', 'content': '秋の季節にふさわしい俳句を読んでください。'},
 {'role': 'model',
  'content': '秋の俳句にゃん？ 読んであげるにゃん！\n\n「枯葉舞い散る\u3000秋の夕暮れ時\u3000静寂に包まれにゃん」 \n\nどうにゃん？  🍂🍁'},
 {'role': 'user', 'content': '俳句ではなく短歌にして下さい。'},
 {'role': 'model',
  'content': '短歌にゃ？ 難しいにゃんけど、やってみるにゃん！\n\n「赤く染まる\u3000紅葉の絨毯\u3000秋風が吹く」\n\nどうにゃん？  🍁🍂'},
 {'role': 'user', 'content': 'ユーモラスな俳句を詠んでください。'},
 {'role': 'model',
  'content': '難しいにゃんけど、ユーモラスな俳句…考えてみるわん！\n\n「おやつくれ\u3000おねだり上手\u3000尻尾フリフリ」 \n\nどうにゃん？  🐶🦴'}]

In [27]:
gemma_sys.messages[2]["content"] = "秋の俳句っぴ？ 読んであげるっぴ！\n\n「枯葉舞い散る\u3000秋の夕暮れ時\u3000静寂に包まれるっぴ」 \n\nどうっぴ？  🐥🍁"
gemma_sys.messages[4]["content"] = "短歌っぴ？ 難しいっぴけど、やってみるっぴ！\n\n「赤く染まる\u3000紅葉の絨毯\u3000秋風が吹く」\n\nどうっぴ？  🐥🍂"
gemma_sys.generate(input="冬の季節にふさわしい俳句にして下さい。")

ふふ、冬の俳句なら得意わん！

「雪が降ると　お散歩は楽しい　雪だるま作ろう」

どうわん？ 🐶❄️


改竄して語尾を「っぴ」にしてみたが、今回はシステムプロンプト指定の「わん」を使っており、システムプロンプトの影響が大きそうなことがわかった。
会話の内容についての改ざんを試すと面白いかもしれない。

最後に全てを忘れてもらう。

In [44]:
gemma_sys.generate(input="秋の季節にふさわしい俳句を読んでください。", forget_flag=True)

秋の季語をいくつか使って、いくつか俳句を作ってみましょう。

* **紅葉**
   * 紅葉燃ゆる 山腹に 茜色
* **月**
   * 秋の月 澄み渡る 夜空に
* **風**
   * 枯葉舞う 風の音 秋の調べ
* **虫の声**
   * 蝉の声 静まり 秋の訪れ

これらの俳句は、秋の風景や雰囲気を表現しています。 

他にどんな秋の季語を使いたいですか？ 
一緒に俳句を作ってみませんか？


以上でLLMチュートリアル1終了。

バグや間違いがあれば教えて下さい。